### Import Modules

In [203]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time


sys.path.append("../")
import fitter.data_loader as dl
import fitter.fit_manager as fm
import fitter.fitter as fit
import fitter.special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [204]:
from importlib import reload
reload(dl)
reload(sf)
reload(fit)
reload(fm)

<module 'fitter.fit_manager' from '../fitter/fit_manager.py'>

### Specify fit parameters

In [208]:
p_dict = {
    'order' : {
        'fit' : 'nnnlo', # 'nlo', 'nnlo', 'nnnlo'
        'vol' : 0, # max 10
        'exclude' : [], # put LECs here
        
        # semi-nnlo corrections
        'include_alpha_s' : False,
        'include_log' : False, 
        'include_log2' : False,
        'include_sunset' : False,
        
        # nnnlo corrections
        'include_latt_n3lo' : False,
    }, 
    'fit_type' : 'poly', #'ma', 'ma-ratio', 'xpt', 'xpt-ratio', or 'poly'; experimental: 'simultaneous', 'ma-old'
    'F2' : 'FpiFpi', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)
    'use_bijnens_central_value' : True,
    
    'use_prior' : False,
    'bias_correct' : True,
    #'include_su2_isospin_corrrection' : False,
    'abbrs' : [u'a06m310L', u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    
    'save_results' : False,
    'save_pickles' : False,
    'output_name' : 'fit_results',
}

In [209]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()
phys_point_data = data_loader.get_phys_point_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(fit_type=p_dict['fit_type'], order=p_dict['order']['fit'], F2=p_dict['F2'], 
                  include_log=p_dict['order']['include_log'], include_log2=p_dict['order']['include_log2'], 
                  include_sunset=p_dict['order']['include_sunset'], include_alpha_s=p_dict['order']['include_alpha_s'], 
                  include_latt_n3lo=p_dict['order']['include_latt_n3lo'], include_FV=(p_dict['order']['vol'] > 6), 
                  use_bijnens_central_value=p_dict['use_bijnens_central_value']
             )
    
# Make fit
fit_manager = fm.fit_manager(fit_data, phys_point_data, prior=prior, **p_dict)

if p_dict['make_plots']:
    data_loader.save_plots(
        fm.fit_manager.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+fit_manager.fit_type+'_'+fit_manager.order['fit']
    )
else:
    print(fit_manager)
    
if p_dict['save_results']:
    data_loader.save_fit_info(fit_manager.fit_info,
                              output_name=p_dict['output_name'], save_pickles=p_dict['save_pickles'])

    
    
t1 = time.time()

print("\nTotal time (s): ", t1 - t0, "\n")

Using default prior.
Making fit...
Done!
Time (s):  0.40510082244873047

Model: poly_FpiFpi_nnnlo_bijnens

Fitted/[FLAG] values at physical point:
	F_K / F_pi = 0.896(55) [1.1907(21)]	(delta_su2 = 0.0042(17))

Parameters:
            A_x      -0.71 (94)      [     0.0 (1.0) ]  
            A_k      -1.4 (8.8)      [        0 (10) ]  
            A_p       7.4 (8.6)      [        0 (10) ]  
            A_a         94 (19)      [       0 (100) ]  
           A_aa     -1259 (143)      [      0 (1000) ]  *
           A_ak         53 (98)      [       0 (100) ]  
           A_ap       -296 (51)      [       0 (100) ]  **
           A_kk        166 (48)      [       0 (100) ]  *
           A_kp        244 (55)      [       0 (100) ]  **
           A_pp        280 (38)      [       0 (100) ]  **

Least Square Fit:
  chi2/dof [dof] = 9.2 [17]    Q = 9.6e-25    logGBF = -35.143

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08*,1e-10,1e-10)    (itns/time = 21/0.4)

Error Budget (relative error):


In [214]:
data_loader.get_fit_results('other_collabs')

OrderedDict([('FLAG',
              {'posterior': {},
               'error_budget': {},
               'name': 'FLAG',
               'FK/Fpi_pm': '1.1932(19)'})])

In [210]:
data_loader.save_fit_info(fit_manager.fit_info, output_name='test', save_pickles=False)

Saving...
{'name': array(['poly_FpiFpi_nnlo_bijnens', 'poly_FpiFpi_nnnlo_bijnens'],
      dtype=object), 'FK/Fpi': array(['1.290(46)', 0.896(55)], dtype=object), 'delta_su2': array(['-0.0069(15)', 0.0042(17)], dtype=object), 'logGBF': array([-229.55757344607872, -35.143459258493934], dtype=object), 'chi2/df': array([32.83697724227291, 9.245049922859796], dtype=object), 'Q': array([9.775841134451409e-108, 9.606711303372008e-25], dtype=object), 'vol': array([0, 0], dtype=object), 'po:A_x': array(['-5.86(73)', -0.71(94)], dtype=object), 'po:L_4': array([nan, nan], dtype=object), 'po:L_5': array([nan, nan], dtype=object), 'po:L_1': array([nan, nan], dtype=object), 'po:L_2': array([nan, nan], dtype=object), 'po:L_3': array([nan, nan], dtype=object), 'po:L_6': array([nan, nan], dtype=object), 'po:L_7': array([nan, nan], dtype=object), 'po:L_8': array([nan, nan], dtype=object), 'po:L_9': array([nan, nan], dtype=object), 'po:A_a': array(['-18.8(1.4)', 94(19)], dtype=object), 'po:A_k': array(['

In [112]:
temp = data_loader.get_fit_results('test')
temp

{'poly_FpiFpi_nnnlo_bijnens': {'name': 'poly_FpiFpi_nnnlo_bijnens',
  'FK/Fpi': 0.896(55),
  'delta_su2': 0.0042(17),
  'logGBF': -35.143459258493934,
  'chi2/df': 9.245049922859796,
  'Q': 9.606711303372008e-25,
  'prior': {'A_a': 0(100),
   'A_aa': 0(1000),
   'A_ak': 0(100),
   'A_ap': 0(100),
   'A_k': 0(10),
   'A_kk': 0(100),
   'A_kp': 0(100),
   'A_p': 0(10),
   'A_pp': 0(100),
   'A_x': 0.0(1.0)},
  'posterior': {'A_a': 94(19),
   'A_aa': -1259(143),
   'A_ak': 53(98),
   'A_ap': -296(51),
   'A_k': -1.4(8.8),
   'A_kk': 166(48),
   'A_kp': 244(55),
   'A_p': 7.4(8.6),
   'A_pp': 280(38),
   'A_x': -0.71(94)},
  'phys_point': {'a/w0': 0(0),
   'a': 0(0),
   'L': inf(0),
   'alpha_s': 0(0),
   'mpi': 134.80(30),
   'mk': 494.20(30),
   'mss': 685.78(44),
   'a2DI': 0(0),
   'Fpi': 92.07(57),
   'FK': 109.96(49),
   'w0': 0.175(10),
   'FK/Fpi_pm': 1.1932(19),
   'mju': 134.80(30),
   'mru': 494.20(30),
   'mjs': 494.20(30),
   'mrs': 685.78(44),
   'lam2_chi_kpi': 1.599(12)e+06

In [ ]:
['']

In [50]:
temp = data_loader.get_fit_results('test')
temp

OrderedDict([('poly_FpiFpi_nnnlo_bijnens',
              {'posterior': {'A_c': nan,
                'L_4': nan,
                'L_5': nan,
                'L_1': nan,
                'L_2': nan,
                'L_3': nan,
                'L_6': nan,
                'L_7': nan,
                'L_8': nan,
                'L_9': nan,
                'A_a': '94(19)',
                'A_k': '-1.4(8.8)',
                'A_p': '7.4(8.6)',
                'A_loga': nan,
                'A_aa': '-1259(143)',
                'A_ak': '53(98)',
                'A_ap': '-296(51)',
                'A_kk': '166(48)',
                'A_kp': '244(55)',
                'A_pp': '280(38)'},
               'FK/Fpi': '0.896(55)',
               'delta_su2': '0.0042(17)',
               'logGBF': -35.14345925849393,
               'chi2/df': 9.245049922859796,
               'Q': 9.606711303372006e-25,
               'vol': 0,
               'A_aa.1': '-1259(143)'})])

In [13]:
import yaml
def test(fit_info):
    output = {}
    output['name'] = fit_info
    
test(fit_manager.fit_info)

{'name': 'poly_FpiFpi_nnnlo_bijnens',
 'FK/Fpi': 0.896(55),
 'delta_su2': 0.0042(17),
 'logGBF': -35.143459258493934,
 'chi2/df': 9.245049922859796,
 'Q': 9.606711303372008e-25,
 'vol': 0,
 'phys_point': {'a/w0': 0(0),
  'a': 0(0),
  'L': inf(0),
  'alpha_s': 0(0),
  'mpi': 134.80(30),
  'mk': 494.20(30),
  'mss': 685.78(44),
  'a2DI': 0(0),
  'Fpi': 92.07(57),
  'FK': 109.96(49),
  'w0': 0.175(10),
  'FK/Fpi_pm': 1.1932(19),
  'mju': 134.80(30),
  'mru': 494.20(30),
  'mjs': 494.20(30),
  'mrs': 685.78(44),
  'lam2_chi_kpi': 1.599(12)e+06,
  'lam2_chi_pipi': 1.338(16)e+06,
  'lam2_chi_kk': 1.909(17)e+06,
  'lam2_chi_00': 1.3653(21)e+06,
  'lam2_chi': 1.338(16)e+06},
 'error_budget': {'disc': 0.005806157075131084,
  'chiral': 0.02401044357312078,
  'pp_input': 0.03649215323344931,
  'stat': 0.033557539215212255},
 'prior': {'A_a': 0(100),
  'A_aa': 0(1000),
  'A_ak': 0(100),
  'A_ap': 0(100),
  'A_c': 0.0(1.0),
  'A_k': 0(10),
  'A_kk': 0(100),
  'A_kp': 0(100),
  'A_p': 0(10),
  'A_pp

In [12]:
import yaml

with open('tmp.yaml', 'w') as file:
    yaml.dump(fit_manager.fit_info, file)
fit_manager.fit_info

{'name': 'poly_FpiFpi_nnnlo_bijnens',
 'FK/Fpi': 0.896(55),
 'delta_su2': 0.0042(17),
 'logGBF': -35.143459258493934,
 'chi2/df': 9.245049922859796,
 'Q': 9.606711303372008e-25,
 'vol': 0,
 'phys_point': {'a/w0': 0(0),
  'a': 0(0),
  'L': inf(0),
  'alpha_s': 0(0),
  'mpi': 134.80(30),
  'mk': 494.20(30),
  'mss': 685.78(44),
  'a2DI': 0(0),
  'Fpi': 92.07(57),
  'FK': 109.96(49),
  'w0': 0.175(10),
  'FK/Fpi_pm': 1.1932(19),
  'mju': 134.80(30),
  'mru': 494.20(30),
  'mjs': 494.20(30),
  'mrs': 685.78(44),
  'lam2_chi_kpi': 1.599(12)e+06,
  'lam2_chi_pipi': 1.338(16)e+06,
  'lam2_chi_kk': 1.909(17)e+06,
  'lam2_chi_00': 1.3653(21)e+06,
  'lam2_chi': 1.338(16)e+06},
 'error_budget': {'disc': 0.005806157075131084,
  'chiral': 0.02401044357312078,
  'pp_input': 0.03649215323344931,
  'stat': 0.033557539215212255},
 'prior': {'A_a': 0(100),
  'A_aa': 0(1000),
  'A_ak': 0(100),
  'A_ap': 0(100),
  'A_c': 0.0(1.0),
  'A_k': 0(10),
  'A_kk': 0(100),
  'A_kp': 0(100),
  'A_p': 0(10),
  'A_pp

In [18]:
print(fit_manager.fmt_error_budget())

Partial Errors:
                  FK/Fpi
------------------------
          A_c:   0.00001
     phys:mpi:   0.00050
      phys:mk:   0.00185
phys:lam2_chi:   0.01729
         stat:   0.00098
------------------------
        total:   0.01742



In [34]:
fit_data = {
    'mpi' : 135.,
    'mk' : 495.5,
    'Fpi' : 92.2,
    'FK' : 110.5,
    'L' : np.inf, 
    'a/w0' : 0,
}
fit_data['lam2_chi'] = (4 *np.pi *fit_data['Fpi'])**2

fit_parameters = {
    'L_1' : 0.000372,
    'L_2' : 0.000493,
    'L_3' : -0.003070,
    'L_4' : 0.000089,
    'L_5' : 0.000377,
    'L_6' : 0.000011,
    'L_7' : -0.000340,
    'L_8' : 0.000294,
    'A_a' : 0.1,
    'A_k' : -3.000000,
    'A_p' : 4.000000
}


fit_manager.fk_fpi_fit_fcn(fit_data=fit_data, fit_parameters=fit_parameters, debug=True)

NameError: name 'bootstrapper' is not defined